In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
import math
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
root.gROOT.SetBatch(True)

In [13]:
root.gSystem.Load("input/WriteEvent_C.so")

1

In [14]:
# Switch: True for HELIOS flat-pt, False for thermal
DO_PT_REWEIGHT = True

In [ ]:
file_path = "/media/yoren/T7_Shield/PHENIX/Dielectrons/helios/"
# ---- choose your input ----
fname = "star_jpsi_ee.root"      # thermal sample (no pt reweight)
outfilename = "output/hists/"+fname.replace(".root", "_hists.root")
print(outfilename)
# fname = "kek.root"        # helios flat-pt sample (needs pt reweight)

In [16]:
f = root.TFile.Open(file_path+fname, "READ")
assert f and not f.IsZombie(), f"Could not open {fname}"

T = f.Get("T")
assert T, "Could not find TTree named 'T'"

print("Entries:", T.GetEntries())
T.Print()

Entries: 10000000
******************************************************************************
*Tree    :T         : Tree with my events                                    *
*Entries : 10000000 : Total =      3201031419 bytes  File  Size =  747573753 *
*        :          : Tree compression factor =   4.28                       *
******************************************************************************
*Branch  :MyEvent                                                            *
*Entries : 10000000 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :fUniqueID : UInt_t                                                 *
*Entries : 10000000 : Total  Size=   40009532 bytes  File Size  =     184815 *
*Baskets :       99 : Basket Size=     879104 bytes  Compression= 216.47     *
*............................................................................*
*Br    1 :fBits     : UInt_t      

In [17]:
def get_tracks_from_event(MyEvent):
    """
    Returns a Python list of track objects (WriteTrack) stored in MyEvent.

    You may need to adapt:
      - MyEvent.GetNEntries()
      - MyEvent.GetEntry(i) or MyEvent.GetTrack(i)
    """
    tracks = []

    # --- ADAPT HERE if needed ---
    n = MyEvent.GetNEntries()   # common
    for i in range(n):
        trk = MyEvent.GetEntry(i)  # common: returns a WriteTrack (or reference)
        tracks.append(trk)

    return tracks

In [18]:
def track_info(trk):
    """
    Returns dict with:
      id, isFinal, br, parent_index, weight, TLorentzVector p4
    Adapt method names if needed.
    """
    # --- ADAPT HERE if needed ---
    pid     = int(trk.GetID())
    isFinal = int(trk.GetFinal())
    br      = float(trk.GetBranch())
    w       = float(trk.GetWeight())

    px = float(trk.GetPx())
    py = float(trk.GetPy())
    pz = float(trk.GetPz())
    en = float(trk.GetEnergy())

    p4 = root.TLorentzVector(px, py, pz, en)

    return {"pid": pid, "isFinal": isFinal, "br": br, "w": w, "p4": p4}

In [19]:
# Example target spectrum (REPLACE with your real one)
T_eff = 0.3  # GeV

def pt_weight(pt):
    """Target weight for helios flat-pt generation."""
    return math.exp(-pt / T_eff)

In [23]:
h2_m_pt = root.TH2D(
    "h2_m_pt",
    "e^{+}e^{-}: M_{ee} vs p_{T};p_{T} (GeV/c);M_{ee} (GeV/c^{2})",
    200, 0.0, 10.0,
    200, 0.0, 5.0
)

h_mult_final = root.TH1D("h_mult_final", "Final-state multiplicity;N_{final};Events", 20, -0.5, 19.5)
h_br_final   = root.TH1D("h_br_final",   "Final track branch br;br;Counts",            50, -0.5, 49.5)

In [ ]:
nentries = T.GetEntries()

for iev in range(nentries):
    T.GetEntry(iev)
    MyEvent = T.MyEvent

    ntrk = int(MyEvent.GetNEntries())

    finals = []
    for i in range(ntrk):
        trk = MyEvent.GetWriteTrack(i)  # by value (copy)
        if int(trk.GetFinal()) != 1:
            continue

        pid = int(trk.GetID())
        br  = float(trk.GetBranch())
        wt  = float(trk.GetWeight())

        p4 = root.TLorentzVector(
            float(trk.GetPx()),
            float(trk.GetPy()),
            float(trk.GetPz()),
            float(trk.GetEnergy())
        )

        finals.append((pid, br, wt, p4))
        h_br_final.Fill(br)

    h_mult_final.Fill(len(finals))

    # pick electrons and positrons among finals
    eminus = [t for t in finals if t[0] == 11]
    eplus  = [t for t in finals if t[0] == -11]

    # form all e- e+ pairs in event (usually 1x1)
    for (pid1, br1, w1, p41) in eminus:
        for (pid2, br2, w2, p42) in eplus:
            pair = p41 + p42
            mee  = pair.M()
            pt   = pair.Pt()

            w = 1.0
            # include HELIOS-provided weights (branching weights)
            w *= (w1 * w2)

            # apply pt reweight only for HELIOS flat-pt generation
            if DO_PT_REWEIGHT:
                w *= pt_weight(pt)

            h2_m_pt.Fill(pt, mee, w)

    if iev > 0 and (iev % 1000000 == 0):
        print("Processed", iev, "/", nentries)

Processed 200000 / 10000000
Processed 400000 / 10000000
Processed 600000 / 10000000
Processed 800000 / 10000000
Processed 1000000 / 10000000
Processed 1200000 / 10000000
Processed 1400000 / 10000000
Processed 1600000 / 10000000
Processed 1800000 / 10000000
Processed 2000000 / 10000000
Processed 2200000 / 10000000
Processed 2400000 / 10000000
Processed 2600000 / 10000000
Processed 2800000 / 10000000
Processed 3000000 / 10000000
Processed 3200000 / 10000000
Processed 3400000 / 10000000
Processed 3600000 / 10000000
Processed 3800000 / 10000000
Processed 4000000 / 10000000
Processed 4200000 / 10000000
Processed 4400000 / 10000000
Processed 4600000 / 10000000
Processed 4800000 / 10000000
Processed 5000000 / 10000000
Processed 5200000 / 10000000
Processed 5400000 / 10000000


In [ ]:
c1 = root.TCanvas("c1","c1",900,700)
h2_m_pt.Draw("COLZ")
c1.SetLogz(True)
c1.SaveAs("mee_vs_pt.png")

c2 = root.TCanvas("c2","c2",800,600)
h_mult_final.Draw()
c2.SaveAs("mult_final.png")

c3 = root.TCanvas("c3","c3",800,600)
h_br_final.Draw()
c3.SaveAs("br_final.png")

In [ ]:
out = root.TFile(outfilename, "RECREATE")
h2_m_pt.Write()
h_mult_final.Write()
h_br_final.Write()
out.Close()

print("Wrote pair_hists.root")